# 영주시 데이터 분석 활용 공모전 - 데이터 전처리

- 코드 참고; https://github.com/wow-kim/COMPAS_SUWON_3rd/blob/master/COMPAS_%EC%B5%9C%EC%A2%85%EC%BD%94%EB%93%9C.ipynb

# 1. 라이브러리 불러오기

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import json
import random
import matplotlib.pylab as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
# 주소로 위도, 경도 크롤링하는 코드
import requests

# 주소로 위도 경도로 구하는 함수
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    headers = {"Authorization": "KakaoAK YOUR_API_KEY"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    try:
      crd = api_json['documents'][0]['address']
    except:
      crd = {'x':None,'y':None}
    return crd

def get_xy(df):
  x_lst = [] # 위도
  y_lst = [] # 경도
  print('********위도, 경도 크롤링 시작 *******')
  for addr in tqdm(df['주소']):
    xy = get_location(addr)
    x = xy['x']
    y = xy['y']
    x_lst.append(x)
    y_lst.append(y)
  df['x'] = x_lst
  df['y'] = y_lst
  return df

# 2. 데이터 불러오기

1. 영주시 기본 정보

  1) base_yj : 법정 경계 https://github.com/vuski/admdongkor/blob/master/ver20230701/HangJeongDong_ver20230701.geojson

  2) foot_traffic : 유동인구 - 민간데이터 활용

2. 교통

  1) bus_station : 영주시 버스정류장 현황


3. 관광
  
  1) cultural_heritage: 문화재 - 공공데이터포털

  2) rural_tour: 농촌 관광 시설 - 공공데이터포털

  3) restaurant : 영주 맛집 - 공공데이터포털


4. 관광 X

  1) village_hall : 마을 회관 - 공공데이터포털

  2) senior_center : 경로당 - 공공데이터포털

  3) public_housing: 공공주택 정보 - 공공데이터포털

  4) public_facility: 다중 이용시설 - 공공데이터포털
  
  5) senior_welfare : 노인 복지시설 - 공공데이터포털

In [ ]:
## 로드가 오래걸림

# 법정 경계
base = json.load(open('/content/drive/MyDrive/영주시_데이터/HangJeongDong_ver20230701.geojson',encoding='utf-8'))

# 유동인구
jan = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202201.csv',encoding='utf-8')
feb = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202202.csv',encoding='utf-8')
mar = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202203.csv',encoding='utf-8')
apr = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202204.csv',encoding='utf-8')
may = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202205.csv',encoding='utf-8')
jun = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202206.csv',encoding='utf-8')
jul = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202207.csv',encoding='utf-8')
aug = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202208.csv',encoding='utf-8')
sep = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202209.csv',encoding='utf-8')
oct = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202210.csv',encoding='utf-8')
nov = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202211.csv',encoding='utf-8')
dec = pd.read_csv('/content/drive/MyDrive/영주시_데이터/50CELL_YEONGJU_202212.csv',encoding='utf-8')

In [ ]:
#버스정류장
bus_station = pd.read_csv('/content/drive/MyDrive/영주시_데이터/경상북도 영주시_버스정류장 현황_20200814 (1).csv',encoding='cp949')

In [ ]:
# 관광사업체 인허가 현황

# 문화재
cultural_heritage = pd.read_csv('/content/drive/MyDrive/영주시_데이터/경상북도 영주시 문화재 정보_20170501. (1).csv',encoding='cp949')

# 농촌관광시설
rural_tour = pd.read_csv('/content/drive/MyDrive/영주시_데이터/경상북도 영주시_농촌관광시설 현황.csv', encoding='utf-8')

# 영주 맛집
restaurant = pd.read_csv('/content/drive/MyDrive/영주시_데이터/경상북도 영주시_영주맛집 현황.csv',encoding='utf-8')

In [ ]:
# 마을 회관
village_hall = pd.read_csv('/content/drive/MyDrive/영주시_데이터/경상북도 영주시_마을회관_20170512..csv',encoding='cp949')

# 경로당
senior_center = pd.read_csv('/content/drive/MyDrive/영주시_데이터/경상북도 영주시_경로당_20170501..csv',encoding='cp949')

# 공동주택 정보
public_housing = pd.read_csv('/content/drive/MyDrive/영주시_데이터/경상북도 영주시_공동주택 정보_20180831..csv',encoding='cp949')

# 다중이용시설
public_facility = pd.read_csv('/content/drive/MyDrive/영주시_데이터/경상북도 영주시_다중이용시설 현황_20181213.csv',encoding='cp949')

# 노인복지시설
senior_welfare = pd.read_csv('/content/drive/MyDrive/영주시_데이터/경상북도 영주시_노인복지시설 현황.csv',encoding='utf-8')

# 3. 데이터 전처리

## 1-1. 법정경계 (base_yj)
- 영주시 행정동 정보만 base_yj 변수에 리스트 형태로 저장

In [ ]:
base_yj = []
for features in base['features']:
  if features['properties']['sggnm']=='영주시':
    base_yj.append(features)

NameError: name 'base' is not defined

In [ ]:
base_yj

[]

## 1-2. 유동인구 (foot_traffic)
- 행정동 코드에 맞게 행정동 column 추가 (법정경계 데이터의 feature name과 동일하게 열 이름을 'temp'로 지정)
- timezn_cd,etl_ymd 열 드랍
- id(=좌푯값 key)를 키로 사용하여 전체 월에 대한 평균값 구함



In [ ]:
# 행정동 추가 및 기존 id 삭제 함수
def hjd(df):
  admi_cd = { 47210250:'풍기읍',
              47210310:'이산면',
              47210320:'평은면',
              47210330:'문수면',
              47210340:'장수면',
              47210350:'안정면',
              47210360:'봉현면',
              47210370:'순흥면',
              47210380:'단산면',
              47210390:'부석면',
              47210510:'상망동',
              47210525:'하망동',
              47210550:'영주1동',
              47210560:'영주2동',
              47210590:'휴천1동',
              47210600:'휴천2동',
              47210610:'휴천3동',
              47210620:'가흥1동',
              47210630:'가흥2동' }
  temp = []
  for row in df['admi_cd']:
      temp.append(admi_cd[row])
  df['temp'] = temp
  df.drop(colums=['admi_cd'],inpace=True)
  return df

# df concat, 불필요한 열 삭제
months = [jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec]
foot_traffic = pd.concat(months)
foot_traffic.drop(columns = ['timezn_cd','etl_ymd'],inplace=True)

# id를 key로 값을 평균내고 hjd 함수 apply
foot_traffic = hjd(foot_traffic.groupby('id').mean())

In [ ]:
foot_traffic

## 2-1. 버스정류장 (bus_station)
- 위도, 경도 열 이름을 각각 x,y로 변경 (다른 데이터와 통일)
- 정류장 ID 열 삭제

In [ ]:
bus_station.drop(columns=['정류장ID'],inplace=True)
bus_station.rename(columns = {'위도':'x','경도':'y','정류장명':'명칭'},inplace=True)
bus_station['시설명']='버스정류장'
bus_station

,명칭,x,y,시설명
0,(구)읍사무소,36.870388,128.524575,버스정류장
1,SK주유소앞,36.832176,128.638726,버스정류장
2,가천,36.801315,128.589752,버스정류장
3,가흥동,36.813366,128.606163,버스정류장
4,가흥동,36.823266,128.605155,버스정류장
...,...,...,...,...
868,화기리,36.766949,128.576496,버스정류장
869,화성라온빌아파트,36.819998,128.635399,버스정류장
870,휴천1동주민센터,36.818592,128.630041,버스정류장
871,희방,36.903878,128.459013,버스정류장


## 3-1. 문화재 (cultural_heritage)

In [ ]:
cultural_heritage.drop(columns=['종별','호수','지정일자','데이터기준일자'],inplace=True)
cultural_heritage.rename(columns={'소재지':'주소'},inplace=True)
cultural_heritage['시설명']='문화재'

In [ ]:
cultural_heritage = get_xy(cultural_heritage)
cultural_heritage

********위도, 경도 크롤링 시작 *******


100%|██████████| 134/134 [01:29<00:00,  1.50it/s]


,명칭,주소,시설명,x,y
0,영주 부석사 무량수전 앞 석등,경상북도 영주시 부석면 복지리 148,문화재,None,None
1,영주 부석사 무량수전,경상북도 영주시 부석면 복지리 148,문화재,None,None
2,영주 부석사 조사당,경상북도 영주시 부석면 복지리 151,문화재,None,None
3,영주 부석사 소조여래좌상,경상북도 영주시 부석면 복지리 148,문화재,None,None
4,부석사조사당벽화,경상북도 영주시 부석면 복지리 148,문화재,None,None
...,...,...,...,...,...
129,영주 안향 향려비,경상북도 영주시 순흥면 석교리 211-1,문화재,128.575418789711,36.9069022691471
130,영주 원리 도율종중 고택,경상북도 영주시 이산면 원리 421,문화재,128.654527161155,36.8216275206308
131,영주 안양원 독성탱화 및 신중탱화,경상북도 영주시 하망동 135-1,문화재,128.627791332931,36.8304366971539
132,영주 구은고택,경상북도 영주시 이산면 신암리 111,문화재,128.693995514531,36.8509737138949


## 3-2. 농촌관광시설 (rural_tour)

In [ ]:
rural_tour.drop(columns= ['TELNO'],inplace=True)
rural_tour.rename(columns={'TRRSRT_NM':'명칭','RDNMADR':'주소'},inplace=True)
rural_tour['시설명']='농촌관광시설'

In [ ]:
rural_tour = get_xy(rural_tour)
rural_tour

********위도, 경도 크롤링 시작 *******


100%|██████████| 15/15 [00:10<00:00,  1.49it/s]


,명칭,주소,시설명,x,y
0,소수서원,경상북도 영주시 순흥면 소백로 2740,농촌관광시설,128.579069285577,36.9252494012094
1,부석사,경상북도 영주시 부석면 부석사로 345,농촌관광시설,128.685645109057,36.9973813281608
2,소백산국립공원,경상북도 영주시 풍기읍 소백로 1794,농촌관광시설,None,None
3,선비촌,경상북도 영주시 순흥면 소백로 2796,농촌관광시설,128.58343367327,36.9284735311732
4,희방사,경상북도 영주시 풍기읍 죽령로1720번길 278,농촌관광시설,128.456719470324,36.9199955969656
5,풍기온천리조트,경상북도 영주시 풍기읍 죽령로 1400,농촌관광시설,128.479058899287,36.8882977674344
6,무섬마을,경상북도 영주시 문수면 무섬로 234번길 41,농촌관광시설,128.623393409148,36.7307327788913
7,소백산자락길,경상북도 영주시 풍기읍 일원,농촌관광시설,None,None
8,영주근대역사문화거리,경상북도 영주시 영주동 일원,농촌관광시설,None,None
9,삼판서고택,경상북도 영주시 선비로 181번길 56-1,농촌관광시설,128.615921967067,36.8207365806188


## 3-3. 영주 맛집 (restaurant)

In [ ]:
restaurant.drop(columns = ['REG_NO','TELNO'],inplace=True)
restaurant.rename(columns={'BSSH_NM':'명칭','ADRES':'주소'},inplace=True)
restaurant['시설명']='영주맛집'

In [ ]:
restaurant = get_xy(restaurant)
restaurant

********위도, 경도 크롤링 시작 *******


100%|██████████| 36/36 [00:24<00:00,  1.48it/s]


,명칭,주소,시설명,x,y
0,계림상황삼계탕,경상북도 영주시 대학로 5-1,영주맛집,128.623519789952,36.8101407388603
1,나드리,"경상북도 영주시 중앙로 89, 2층",영주맛집,128.624846310981,36.8233664323827
2,너른마당,경상북도 영주시 구성로 117,영주맛집,128.629549324648,36.8038141755563
3,다성한정식,경상북도 영주시 선비로 128(휴천동),영주맛집,128.622100182565,36.8162578677194
4,대지한우식육식당,경상북도 영주시 선비로 48,영주맛집,128.62501599032,36.8095814955465
5,동화식당,경상북도 영주시 중앙로 81번길 31,영주맛집,128.623285643893,36.8236135459685
6,둘구비가든,경상북도 영주시 조암로 409,영주맛집,128.658432013515,36.8070911039818
7,명동감자탕,"경상북도 영주시 중앙로106번길 8, 1호",영주맛집,128.626207040608,36.8245283238864
8,미소함박,경상북도 영주시 대학로262번길 8-1,영주맛집,128.605699285732,36.8185794143674
9,삼뜨락한정식,경상북도 영주시 풍기읍 안풍로 307-12,영주맛집,128.540904287163,36.8711993203258


## 4-1. 마을회관 (village_hall)

In [ ]:
village_hall.drop(columns = ['마을명칭(자연부락)', '건립연도', '연면적(㎡)', '소재지지번주소','건축면적(㎡)', '담당자연락처', '등기여부', '대표자', '데이터기준일자','담당자소속','이용인구(명)'],inplace=True )
village_hall.rename(columns = {'소재지도로명주소':'주소'},inplace=True)
village_hall['시설명'] = '마을회관'

In [ ]:
vilaage_hall = get_xy(village_hall)
village_hall

********위도, 경도 크롤링 시작 *******


100%|██████████| 47/47 [00:31<00:00,  1.48it/s]


,명칭,주소,시설명,x,y
0,성내1리 마을회관,경상북도 영주시 풍기읍 기주로 132,마을회관,128.521762368904,36.8717950635425
1,성내2리 마을회관,경상북도 영주시 풍기읍 기주로 88-6,마을회관,128.526356336653,36.8703203803483
2,성내3리 마을회관,경상북도 영주시 풍기읍 남원로 86-8,마을회관,128.525169718805,36.8679218958324
3,성내4리 새마을회관,경상북도 영주시 풍기읍 남원로 112,마을회관,128.522844644107,36.8686862211045
4,동부1리 마을회관,경상북도 영주시 풍기읍 기주로65번길 5,마을회관,128.52839359671,36.8688827586613
5,동부2리 새마을회관,경상북도 영주시 풍기읍 기주로60번길 33,마을회관,128.530786617143,36.8691094636044
6,동부6리 마을회관,경상북도 영주시 풍기읍 동성로70번길 17-12,마을회관,128.528867923208,36.871660601964
7,금계1리 공원산 마을회관,경상북도 영주시 풍기읍 기주로170번길 58,마을회관,128.518416252631,36.8762007511219
8,금계1리 마을회관,경상북도 영주시 풍기읍 무릉길2번길 37,마을회관,128.521933081999,36.8867879528963
9,서부1리 마을회관,경상북도 영주시 풍기읍 동성로94번길 20-14,마을회관,128.52682866026,36.8722909513695


## 4-2. 경로당 (senior_center)

In [ ]:
senior_center.drop(columns = ['읍면동','이용인원','노인회장','연락처','데이터기준일자'],inplace=True)
senior_center.rename(columns={'소재지주소':'주소','경로당명':'명칭'},inplace=True)
senior_center['시설명'] = '경로당'

In [ ]:
senior_center = get_xy(senior_center)
senior_center

********위도, 경도 크롤링 시작 *******


100%|██████████| 335/335 [03:42<00:00,  1.50it/s]


,명칭,주소,시설명,x,y
0,교촌1리,경상북도 영주시 풍기읍 금계로10번길 24,경로당,128.527791747851,36.8765289113444
1,교촌2리,경상북도 영주시 풍기읍 동양대로129번길 162,경로당,128.527351827293,36.8845382407621
2,금계1리,경상북도 영주시 풍기읍 무릉길42번길 9,경로당,128.519836317644,36.8846598481687
3,금계2리,경상북도 영주시 풍기읍 금계로 217,경로당,128.527117915352,36.891264127661
4,동부1리,경상북도 영주시 풍기읍 기주로65번길 5,경로당,128.52839359671,36.8688827586613
...,...,...,...,...,...
330,줄포,경상북도 영주시 줄포길 78-4 (상줄동),경로당,128.576578055104,36.8333804965263
331,창진,경상북도 영주시 창진로 195 (창진동),경로당,128.598966656211,36.8424595760244
332,필두,경상북도 영주시 필두길74번길 52 (상줄동),경로당,128.585606083814,36.8325101406165
333,현대아파트,경상북도 영주시 신재로12번길 67 (가흥동),경로당,128.610698634651,36.8339407311954


## 4-3. 공공주택 (public_housing)

In [ ]:
public_housing.drop(columns=['세대수', '소재지지번주소', '층', '동', '준공일자', '사업주체', '관리주체', '데이터기준일자'],inplace=True)
public_housing.rename(columns={'소재지도로명주소':'주소','단지명':'명칭'},inplace=True)
public_housing['시설명']='공동주택'

In [ ]:
public_housing = get_xy(public_housing)
public_housing

********위도, 경도 크롤링 시작 *******


100%|██████████| 96/96 [01:04<00:00,  1.50it/s]


,명칭,주소,시설명,x,y
0,휴천주공,경북 영주시 구성로188번길 27 (휴천동),공동주택,128.630070350736,36.8104471915498
1,동서주택,경북 영주시 원당로207번길 51 (하망동),공동주택,128.633233833892,36.8308771468371
2,보라연립주택,경북 영주시 번영로 65 (휴천동),공동주택,128.628353107122,36.8181161290434
3,영주직업훈련원 기숙사,경북 영주시 가흥로 2 (문정동),공동주택,128.602822953576,36.8007410896618
4,세종주택,경북 영주시 선비로 40 (휴천동),공동주택,128.625303800883,36.809292752781
5,유진연립,경북 영주시 구성로261번길 59 (휴천동),공동주택,128.623977958884,36.8165519096017
6,영주동 주공,경북 영주시 선비로 203 (영주동),공동주택,128.615697476096,36.8225053366314
7,철도연립,경북 영주시 구성로210번길 18-15 (휴천동),공동주택,128.629536555566,36.8121137140078
8,덕흥주택,경북 영주시 구성로 248-1 (휴천동),공동주택,128.628214013775,36.8149203609358
9,동산연립,경북 영주시 대동로215번길 68 (휴천동),공동주택,128.626414049119,36.818550918781


## 4-4. 다중이용시설 (public_facility)

In [ ]:
public_facility.drop(columns=['전화번호','데이터기준일자','시설구분'],inplace=True)
public_facility.rename(columns = {'시설명':'명칭','소재지':'주소'},inplace=True)
public_facility['시설명']='다중이용시설'

In [ ]:
public_facility = get_xy(public_facility)
public_facility

********위도, 경도 크롤링 시작 *******


100%|██████████| 45/45 [00:30<00:00,  1.49it/s]


,명칭,주소,시설명,x,y
0,성누가병원,경상북도 영주시 구성로 329 (영주동),다중이용시설,128.624654478525,36.821391164885
1,새희망병원,경상북도 영주시 반지미로 275-17 (가흥동),다중이용시설,128.595771508582,36.8246820109238
2,영주기독병원,경상북도 영주시 구성로 380 (영주동),다중이용시설,128.620938320857,36.8253056805805
3,(의)영풍의료재단 풍기성심요양병원,경상북도 영주시 풍기읍 소백로1916번길 6,다중이용시설,128.536203406714,36.8674640501926
4,영주가톨릭병원,경상북도 영주시 번영로 176 (영주동),다중이용시설,128.625035389843,36.8267422600575
5,영주시립노인요양병원,경상북도 영주시 안정면 장안로 697,다중이용시설,128.564909839612,36.8301564270377
6,영주삼봉정신병원,경상북도 영주시 꽃동산로 43 (가흥동),다중이용시설,128.619784880669,36.8132551104617
7,(의)기호의료재단 청하요양병원,경상북도 영주시 신재로24번길 91 (가흥동),다중이용시설,128.609025211275,36.8351262346441
8,(의)영주의료재단 명품요양병원,경상북도 영주시 반지미로 265 (가흥동),다중이용시설,128.597597508385,36.824809034954
9,영주자인병원,경상북도 영주시 대동로31번길 9 (가흥동),다중이용시설,128.605597376739,36.8158847899167


## 4-5. 노인복지시설 (senior_welfare)

In [ ]:
senior_welfare.drop(columns = ['SE','SMALL_SE','RPRSNTV','PSNCPA','NOW_NMPR','OPER_MBY','CTTPC','RM','EMP_CO'],inplace=True)
senior_welfare.rename(columns = {'FCLTY_NM':'명칭','RDNMADR':'주소'},inplace=True)
senior_welfare['시설명']='노인복지시설'

In [ ]:
senior_welfare = get_xy(senior_welfare)
senior_welfare

********위도, 경도 크롤링 시작 *******


100%|██████████| 46/46 [00:30<00:00,  1.50it/s]


,명칭,주소,시설명,x,y
0,만수촌,경상북도 영주시 조와로277번길 298,노인복지시설,128.621468520869,36.8712671842348
1,소망의집,경상북도 영주시 조와로277번길 233 234,노인복지시설,None,None
2,인애가장수마을,경상북도 영주시 안정면 장안로 709-25,노인복지시설,128.562107369602,36.8298572999395
3,영주노인전문요양원,경상북도 영주시 반지미로 236-50,노인복지시설,128.590858001594,36.8179373964008
4,이당원,경상북도 영주시 안정면 장안로709-25,노인복지시설,128.562107369602,36.8298572999395
5,평강소규모요양센터,경상북도 영주시 조와로277번길 233,노인복지시설,128.621129430308,36.8704091035113
6,영주소규모요양센터,경상북도 영주시 반지미로 236-50,노인복지시설,128.590858001594,36.8179373964008
7,다락원실버텔,경상북도 영주시 조와로 277번길238,노인복지시설,128.622310285544,36.870287119182
8,새희망실버힐스,경상북도 영주시 반지미로 275-17,노인복지시설,128.595771508582,36.8246820109238
9,성심요양원,경상북도 영주시 풍기읍 소백로1916번길 6,노인복지시설,128.536203406714,36.8674640501926


# 4. 데이터 합치기 및 내보내기

In [ ]:
lst = [bus_station, cultural_heritage, rural_tour, restaurant, village_hall, senior_center, public_housing, public_facility, senior_welfare]

df = pd.concat(lst,join='outer')
df.drop_duplicates(subset='명칭')
df

,명칭,x,y,시설명,주소
0,(구)읍사무소,36.870388,128.524575,버스정류장,NaN
1,SK주유소앞,36.832176,128.638726,버스정류장,NaN
2,가천,36.801315,128.589752,버스정류장,NaN
3,가흥동,36.813366,128.606163,버스정류장,NaN
4,가흥동,36.823266,128.605155,버스정류장,NaN
...,...,...,...,...,...
41,한빛의료기,128.620585231652,36.8250589336187,노인복지시설,경상북도 영주시 구성로 381 (영주동)
42,삼성재가복지센터,128.626926119998,36.8280904314489,노인복지시설,경상북도 영주시 명륜길 5 (영주동)
43,부모마음재가복지센터,None,None,노인복지시설,경상북도 영주시 봉현면 소백로1624번길16-24 B동
44,우리재가노인복지센터,128.606976360575,36.8145152745995,노인복지시설,"경상북도 영주시 대동로34번길 3-8,101호(가흥동)"


In [ ]:
# 저장
df.to_csv('./전처리.csv',encoding='utf-8')

In [ ]:
# 위도, 경도 None인 곳은 직접 추가
df[df.x.isna()]

,명칭,x,y,시설명,주소
0,영주 부석사 무량수전 앞 석등,None,None,문화재,경상북도 영주시 부석면 복지리 148
1,영주 부석사 무량수전,None,None,문화재,경상북도 영주시 부석면 복지리 148
2,영주 부석사 조사당,None,None,문화재,경상북도 영주시 부석면 복지리 151
3,영주 부석사 소조여래좌상,None,None,문화재,경상북도 영주시 부석면 복지리 148
4,부석사조사당벽화,None,None,문화재,경상북도 영주시 부석면 복지리 148
8,영주 영주동 석조여래입상,None,None,문화재,경상북도 영주시 가흥1동 2-69
74,휴천리 선사시대 지석 및 입석,None,None,문화재,경상북도 영주시 휴천1동 226-5
2,소백산국립공원,None,None,농촌관광시설,경상북도 영주시 풍기읍 소백로 1794
7,소백산자락길,None,None,농촌관광시설,경상북도 영주시 풍기읍 일원
8,영주근대역사문화거리,None,None,농촌관광시설,경상북도 영주시 영주동 일원
